In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# --- New Import for Phase 3 ---
from pynput.keyboard import Controller as KeyboardController
from pynput.mouse import Button, Controller as MouseController

In [ ]:
# --- Initialization and Setup ---
mp_hands = mp.solutions.hands 
mp_drawing = mp.solutions.drawing_utils 

# Landmark IDs for Tips (4, 8, 12, 16, 20)
TIP_IDS = [4, 8, 12, 16, 20] 

cap = cv2.VideoCapture(0)

# --- Phase 3: Initialize Input Controllers ---
keyboard = KeyboardController()
mouse = MouseController()

# --- State Variables to Prevent Rapid Input ---
# This dictionary tracks the current state of the gestures to ensure 
# we only register a press when the state *changes* from False to True.
input_state = {
    'is_pinching': False,
    'last_number_pressed': -1 # -1 means no number is currently being "held"
}

# --- Function to Determine which Fingers are Up (from Phase 2) ---
def check_fingers_up(hand_landmarks):
    """
    Checks which fingers (Thumb, Index, Middle, Ring, Pinky) are extended.
    Returns a list of booleans: [is_thumb_up, is_index_up, ..., is_pinky_up]
    """
    fingers_up = []
    landmarks = hand_landmarks.landmark
    
    # 1. Thumb Check (Landmark 4 vs 3 - X-axis movement)
    if landmarks[TIP_IDS[0]].x > landmarks[TIP_IDS[0] - 1].x: 
        fingers_up.append(True)
    else:
        fingers_up.append(False)
        
    # 2. Check other 4 Fingers (Index, Middle, Ring, Pinky) - Y-axis movement
    for i in range(1, 5):
        tip_y = landmarks[TIP_IDS[i]].y
        pip_y = landmarks[TIP_IDS[i] - 2].y 
        
        if tip_y < pip_y:
            fingers_up.append(True)
        else:
            fingers_up.append(False)
            
    return fingers_up

# --- Function to Detect Pinch Gesture (from Phase 2) ---
def detect_pinch(hand_landmarks, h, w):
    """
    Detects if the thumb tip and index finger tip are close enough for a pinch.
    """
    landmarks = hand_landmarks.landmark
    
    thumb_tip_lm = landmarks[TIP_IDS[0]]
    index_tip_lm = landmarks[TIP_IDS[1]]
    
    # Convert normalized coordinates to pixel coordinates
    thumb_x = int(thumb_tip_lm.x * w)
    thumb_y = int(thumb_tip_lm.y * h)
    
    index_x = int(index_tip_lm.x * w)
    index_y = int(index_tip_lm.y * h)
    
    # Calculate Euclidean distance
    distance = np.sqrt((index_x - thumb_x)**2 + (index_y - thumb_y)**2)
    
    PINCH_THRESHOLD = 50 
    
    if distance < PINCH_THRESHOLD:
        return True, (index_x, index_y)
    else:
        return False, None


with mp_hands.Hands(
        static_image_mode=False,
        max_num_hands=1,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:
    
    # --- Main Processing Loop ---
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            continue
            
        h, w, c = image.shape
        
        # Mirror the image horizontally to make controls intuitive (optional)
        image = cv2.flip(image, 1) 
        
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        current_gesture_text = "No Hand Detected"
        current_number = -1 # Reset number

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                
                # A. Finger Counting and Typing Logic
                fingers_up = check_fingers_up(hand_landmarks)
                count = sum(fingers_up)
                current_number = count

                # B. Pinch Gesture Logic (Click Simulation)
                is_pinching, pinch_center = detect_pinch(hand_landmarks, h, w)
                
                
                # --- PHASE 3: INPUT INTEGRATION LOGIC ---
                
                # 1. Pinch Detection and Mouse Click
                if is_pinching and not input_state['is_pinching']:
                    # New pinch detected, simulate a mouse click (e.g., to click a virtual keyboard key)
                    mouse.click(Button.left)
                    input_state['is_pinching'] = True # Set state to prevent rapid clicks
                    current_gesture_text = f"PINCH DETECTED! (CLICKED)"
                    if pinch_center:
                        cv2.circle(image, pinch_center, 15, (0, 0, 255), -1) 
                
                elif not is_pinching and input_state['is_pinching']:
                    # Pinch released
                    input_state['is_pinching'] = False
                
                # 2. Number Guessing and Key Press
                if count >= 1 and count <= 5:
                    current_gesture_text = f"Typing: '{count}'"
                    
                    if count != input_state['last_number_pressed']:
                        # Only press the key if the count has changed or a new gesture is held
                        keyboard.press(str(count))
                        keyboard.release(str(count))
                        input_state['last_number_pressed'] = count
                        
                elif count == 0:
                     # If the hand is closed (0 fingers up), ensure the system registers that no number is held
                     if input_state['last_number_pressed'] != -1:
                         input_state['last_number_pressed'] = -1
                         current_gesture_text = "Hand Closed (Ready for New Input)"
                
                # Draw the standard hand landmarks
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)


        # --- Display Status and Frame ---
        cv2.putText(image, f"Gesture: {current_gesture_text}", (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        cv2.imshow('Hand Tracking Input System - FINAL', image)
        
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

# Release resources
cap.release()
cv2.destroyAllWindows()

e:\projects\hand_track_num_detect\venv\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


KeyboardInterrupt: 

: 

In [ ]:
434353213121543432143212122323254254